In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Proposta para modelagem profunda de dados de mobilidade

    1 - Carregamento de Dados: Você começa carregando dados de um arquivo CSV e escolhe manter apenas as colunas relevantes para a análise.
    2 - Tratamento de Dados:
    2.1 - Limpeza de dados, preenchendo valores ausentes com a média da coluna correspondente. Isso ajuda a manter a consistência dos dados e evita erros durante o treinamento da rede neural.
    2.2 - Codificação de variáveis categóricas usando pd.get_dummies(), que transforma atributos categóricos em numéricos, essencial para processamento por modelos de aprendizado de máquina.
    3 - Seleção de Variáveis
    4 - Divisão dos Dados: Os dados foram divididos em conjuntos de treino e teste, utilizando uma proporção de 80/20, o que é comum em aplicações de aprendizado de máquina.
    5 - Padronização dos Dados: Utilização do StandardScaler para normalizar os dados de entrada, garantindo que a rede neural não seja enviesada por variáveis em diferentes escalas.
    6 - Configuração da Rede Neural: A rede neural foi configurada com três camadas: uma camada de entrada com 64 neurônios, uma camada oculta com 32 neurônios e uma camada de saída com um único neurônio. As funções de ativação relu foram usadas nas camadas intermediárias para introduzir não-linearidade, e linear na camada de saída, apropriada para problemas de regressão.
    7 - Treinamento do Modelo: O modelo foi treinado por 50 épocas com lotes de tamanho 32. Durante o treinamento, os dados de validação são usados para monitorar o desempenho do modelo em dados não vistos durante o treinamento.
    8 - Avaliação do Modelo: O modelo foi avaliado usando o conjunto de teste para calcular o erro quadrático médio (MSE) e o coeficiente de determinação (R²).



In [2]:
# Carregamento dos dados para análise da extrutura
base_tratada_caminho = '..\\Base de dados\\Base_tratada.csv'

# Leitura dos arquivos CSV de acidentes e empreendimentos
base_tratada_df = pd.read_csv(base_tratada_caminho)
pd.set_option('display.max_columns', None)
base_tratada_df

C:\Users\kaiog\AppData\Local\Temp\ipykernel_34144\718623421.py:5: DtypeWarning: Columns (43,63) have mixed types. Specify dtype option on import or set low_memory=False.
  base_tratada_df = pd.read_csv(base_tratada_caminho)


,Unnamed: 0,cod_mdr,cod_operacao,cod_saci,cod_convenio_siafi,cod_ag_operador,origem,dsc_concedente,dsc_fonte,dsc_unidade,cod_proposta,dsc_situacao_contrato_mdr,bln_carteira_mdr,bln_carteira_mdr_ativo,dsc_situacao_objeto_mdr,dsc_paralisada_mdr,bln_ativo,bln_ativo_boolean,cod_id_fonte,cod_id_pac,programa,acao,uf_SIGLA_UF_x,mun_MUNNOMEX_x,Código IBGE,empreendimento,vlr_repasse_financiamento,vlr_contrapartida,vlr_empenhado,vlr_desembolsado,vlr_desbloqueado,vlr_desembolsado_19,vlr_desembolsado_20,vlr_desembolsado_21,situacao_obra,situacao_obra_base_validacao,situacao_contrato,prc_fisico,dte_assinatura,dte_inicio_obra,dte_fim_obra,dte_ult_desembolso,dte_ult_desbloqueio,dte_ult_ref_bm,vlr_ult_desembolso,vlr_ult_repasse_desbloqueado,vlr_ult_contrapartida_desbloqueado,agente_financeiro,agente_financeiro_saci,dias_paralisados,situacao_obra_base_validacao_secex,situacao_obra_mdr,pop_beneficiada,emp_gerado,vlr_investimento,vlr_ult_investimento_desbloqueado,vlr_repasse_financiamento_num,vlr_contrapartida_num,vlr_investimento_num,vlr_empenhado_num,vlr_desembolsado_num,vlr_desbloqueado_num,vlr_desembolsado_19_num,vlr_desembolsado_20_num,vlr_desembolsado_21_num,vlr_ult_desembolso_num,vlr_ult_repasse_desbloqueado_num,vlr_ult_contrapartida_desbloqueado_num,vlr_ult_investimento_desbloqueado_num,ano_assinatura,ano_inicio_obra,ano_fim_obra,ano_ult_desembolso,ano_ult_desbloqueio,ano_ult_ref_bm,ano_x,mun_MUNNOME_x,Populacao_x,mun_AMAZONIA_x,mun_FRONTEIRA_x,mun_CAPITAL_x,uf_CODIGO_UF_x,mun_LATITUDE_x,mun_LONGITUDE_x,mun_ALTITUDE_x,mun_AREA_x,mun_codigo_adotado_x,uf_NOME_UF_x,Região_x,uf_REGIAO_x,Município_x,mun_coordenadas_x,Categoria_emprendimentos,ano_y,Região_y,Município_y,Pedestre_mortes,Ciclista_mortes,Motociclista_mortes,Ocup_triciclo_motor_mortes,Ocup_automovel_mortes,Ocup_caminhonete_mortes,Ocup_veic_transp_pesado_mortes,Ocup_onibus_mortes,Outros_mortes,Populacao_y,total_mortes,taxa_mun_mortes,taxa_mun_pedestre_mortes,taxa_mun_ciclista_mortes,taxa_mun_motociclista_mortes,taxa_mun_automovel_mortes,taxa_mun_veiculo_pesado_mortes,taxa_mun_onibus_mortes,taxa_mun_outros_mortes,mun_MUNNOME_y,mun_MUNNOMEX_y,mun_AMAZONIA_y,mun_FRONTEIRA_y,mun_CAPITAL_y,mun_LATITUDE_y,mun_LONGITUDE_y,mun_ALTITUDE_y,mun_AREA_y,mun_codigo_adotado_y,uf_SIGLA_UF_y,uf_CODIGO_UF_y,uf_NOME_UF_y,uf_REGIAO_y,mun_coordenadas_y,Pedestre_feridos,Ciclista_feridos,Motociclista_feridos,Ocup_triciclo_motor_feridos,Ocup_automovel_feridos,Ocup_caminhonete_feridos,Ocup_veic_transp_pesado_feridos,Ocup_onibus_feridos,Outros_feridos,total_feridos,taxa_mun_feridos,taxa_mun_pedestre_feridos,taxa_mun_ciclista_feridos,taxa_mun_motociclista_feridos,taxa_mun_automovel_feridos,taxa_mun_veiculo_pesado_feridos,taxa_mun_onibus_feridos,taxa_mun_outros_feridos
0,0,1-38001,319910.0,38001.0,0.0,375658.0,PAC_MCID,MINISTERIO DAS CIDADES,FIN,SEMOB,NaN,CONTRATADO - CONCLUIDO,SIM,NAO,CONCLUIDA,NAO PARALISADA,S,Y,375658,38001,PAC FIN,FGTS,PE,RECIFE,2611606.0,IMPLANTACAO DO TRANSPORTE RAPIDO POR ONIBUS NO...,62211677.65,15708716.86,NaN,62211677.65,62211677.65,2339532.99,NaN,NaN,CONCLUIDA,CONCLUIDA,EM SITUAÇÃO NORMAL,100.0,2011-12-26,2011-12-01,2021-11-01,2019-08-01,2019-08-01,NaN,305872.53,0.0,0.0,CAIXA ECONOMICA FEDERAL - MATRIZ,CAIXA,1382.0,CONCLUÍDA A PARTIR 2019,CONCLUÍDA,77342,1437,77920394.51,0.0,62200000.0,15700000.0,77900000.0,0.0,62200000.0,62200000.0,2300000.0,NaN,0.0,305900000.0,0.0,0.0,0.0,2011.0,2011.0,2021.0,2019.0,2019.0,NaN,2011.0,Recife,1546516.0,N,N,S,26.0,-8.054,-34.881,4.0,217494.0,261160.0,PERNAMBUCO,NE,Nordeste,Recife,"-8.054,-34.881",Transporte Público,2010,NE,Recife,242,14,200,0,45,0,4,1,174,1537704,680.0,44.221800,15.7377,0.9104,13.0064,2.9264,0.2601,0.0650,11.315600,Recife,RECIFE,N,N,S,-8.054,-34.881,4.0,217.494,261160,PE,26,PERNAMBUCO,Nordeste,"-8.05400,-34.88100",35,2,13,0,7,1,2,0,1,61.0,3.9670,2.2761,0.1301,0.8454,0.5203,0.1301,0.0000,0.0650
1,1,1-38001,319910.0,38001.0,0.0,375658.0,PAC_MCID,MINISTERIO DAS CIDADES,FIN,SEMOB,NaN,CONTRATADO - CONCLUIDO,SIM,NAO,CONCLUIDA,NAO PARALISAD

In [3]:
# Filtrar colunas relevantes
colunas_relevantes = ['origem', 'dsc_fonte', 'dsc_unidade', 'dsc_situacao_contrato_mdr','uf_SIGLA_UF_x','mun_MUNNOMEX_x',
'vlr_repasse_financiamento','vlr_contrapartida','vlr_desembolsado','vlr_desbloqueado','situacao_obra','vlr_ult_desembolso',
'vlr_ult_repasse_desbloqueado','vlr_ult_contrapartida_desbloqueado','dias_paralisados','emp_gerado','vlr_investimento','vlr_repasse_financiamento_num',
'vlr_contrapartida_num','vlr_investimento_num','vlr_empenhado_num','vlr_desbloqueado_num','Populacao_x','mun_AREA_x','Categoria_emprendimentos',
'Pedestre_mortes','Ciclista_mortes','Motociclista_mortes','Ocup_triciclo_motor_mortes','Ocup_automovel_mortes','Ocup_caminhonete_mortes','Ocup_veic_transp_pesado_mortes',
'Ocup_onibus_mortes','Outros_mortes','total_mortes','taxa_mun_mortes','taxa_mun_pedestre_mortes','taxa_mun_ciclista_mortes','taxa_mun_motociclista_mortes',
'taxa_mun_automovel_mortes','taxa_mun_veiculo_pesado_mortes','taxa_mun_onibus_mortes','taxa_mun_outros_mortes','Pedestre_feridos','Ciclista_feridos','Motociclista_feridos',
'Ocup_triciclo_motor_feridos','Ocup_automovel_feridos','Ocup_caminhonete_feridos','Ocup_veic_transp_pesado_feridos','Ocup_onibus_feridos','Outros_feridos','total_feridos',
'taxa_mun_feridos','taxa_mun_pedestre_feridos','taxa_mun_ciclista_feridos','taxa_mun_motociclista_feridos','taxa_mun_automovel_feridos','taxa_mun_veiculo_pesado_feridos',
'taxa_mun_onibus_feridos','taxa_mun_outros_feridos']

# Filtrar o DataFrame para manter apenas as colunas relevantes
df_filtrado = base_tratada_df[colunas_relevantes]
df_filtrado.dropna()
df_filtrado['vlr_desembolsado'].fillna(df_filtrado['vlr_desembolsado'].mean(), inplace=True)
df_filtrado['vlr_desbloqueado'].fillna(df_filtrado['vlr_desbloqueado'].mean(), inplace=True)
df_filtrado['vlr_ult_desembolso'].fillna(df_filtrado['vlr_ult_desembolso'].mean(), inplace=True)
df_filtrado['vlr_ult_repasse_desbloqueado'].fillna(df_filtrado['vlr_ult_repasse_desbloqueado'].mean(), inplace=True)
df_filtrado['dias_paralisados'].fillna(df_filtrado['dias_paralisados'].mean(), inplace=True)
df_filtrado['Populacao_x'].fillna(df_filtrado['Populacao_x'].mean(), inplace=True)
df_filtrado['mun_AREA_x'].fillna(df_filtrado['mun_AREA_x'].mean(), inplace=True)
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376754 entries, 0 to 376753
Data columns (total 61 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   origem                              376754 non-null  object 
 1   dsc_fonte                           376754 non-null  object 
 2   dsc_unidade                         376754 non-null  object 
 3   dsc_situacao_contrato_mdr           376754 non-null  object 
 4   uf_SIGLA_UF_x                       376754 non-null  object 
 5   mun_MUNNOMEX_x                      375154 non-null  object 
 6   vlr_repasse_financiamento           376754 non-null  float64
 7   vlr_contrapartida                   376754 non-null  float64
 8   vlr_desembolsado                    376754 non-null  float64
 9   vlr_desbloqueado                    376754 non-null  float64
 10  situacao_obra                       376704 non-null  object 
 11  vlr_ult_desembolso        

C:\Users\kaiog\AppData\Local\Temp\ipykernel_34144\1679973764.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['vlr_desembolsado'].fillna(df_filtrado['vlr_desembolsado'].mean(), inplace=True)
C:\Users\kaiog\AppData\Local\Temp\ipykernel_34144\1679973764.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['vlr_desbloqueado'].fillna(df_filtrado['vlr_desbloqueado'].mean(), inplace=True)
C:\Users\kaiog\AppData\Local\Temp\ipykernel_34144\1679973764.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [4]:
# Codificar todas as colunas categóricas usando get_dummies
df_encoded = pd.get_dummies(df_filtrado, columns=['origem', 'dsc_fonte', 'dsc_unidade', 'dsc_situacao_contrato_mdr', 
                                                  'uf_SIGLA_UF_x', 'mun_MUNNOMEX_x', 'situacao_obra', 'Categoria_emprendimentos'])

In [6]:
# Selecionar variáveis relevantes para prever total de mortes
features = ['origem_NPAC_DBGESTORES', 'dsc_unidade_SEMOB', 'dsc_fonte_OGU', 'situacao_obra_OBJETO CONCLUÍDO',
            'Categoria_emprendimentos_Pavimentação, Drenagem e Saneamento','Pedestre_mortes','Motociclista_mortes',
           'Ocup_automovel_mortes','total_feridos']
target = 'total_mortes'
# Criar os conjuntos de treino e teste
X = df_encoded[features]
y = df_encoded[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Padronizar os dados para treinamento
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
# Criar os conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [9]:
# Configurar o modelo de rede neural
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

# Compilar o modelo
model.compile(optimizer='adam', loss='mean_squared_error')

C:\Users\kaiog\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Treinar o modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
9419/9419 ━━━━━━━━━━━━━━━━━━━━ 7s 705us/step - loss: 926.0212 - val_loss: 129.1015
Epoch 2/50
9419/9419 ━━━━━━━━━━━━━━━━━━━━ 7s 700us/step - loss: 140.4439 - val_loss: 119.3463
Epoch 3/50
9419/9419 ━━━━━━━━━━━━━━━━━━━━ 6s 685us/step - loss: 120.8501 - val_loss: 104.2555
Epoch 4/50
9419/9419 ━━━━━━━━━━━━━━━━━━━━ 7s 700us/step - loss: 112.2106 - val_loss: 98.9635
Epoch 5/50
9419/9419 ━━━━━━━━━━━━━━━━━━━━ 6s 687us/step - loss: 106.2453 - val_loss: 95.7377
Epoch 6/50
9419/9419 ━━━━━━━━━━━━━━━━━━━━ 6s 686us/step - loss: 99.5488 - val_loss: 96.0199
Epoch 7/50
9419/9419 ━━━━━━━━━━━━━━━━━━━━ 6s 684us/step - loss: 96.6737 - val_loss: 110.7315
Epoch 8/50
9419/9419 ━━━━━━━━━━━━━━━━━━━━ 7s 692us/step - loss: 94.1270 - val_loss: 89.5083
Epoch 9/50
9419/9419 ━━━━━━━━━━━━━━━━━━━━ 7s 692us/step - loss: 90.5218 - val_loss: 88.2592
Epoch 10/50
9419/9419 ━━━━━━━━━━━━━━━━━━━━ 6s 662us/step - loss: 88.2445 - val_loss: 91.8905
Epoch 11/50
9419/9419 ━━━━━━━━━━━━━━━━━━━━ 6s 671us/step - loss: 84.74

In [11]:
# Fazer previsões com o conjunto de teste
y_pred_nn = model.predict(X_test)

2355/2355 ━━━━━━━━━━━━━━━━━━━━ 1s 465us/step


In [13]:
# Avaliar a performance do modelo
mse_nn = mean_squared_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)
print("Erro quadrático médio: ", mse_nn,"\nR2: ", r2_nn)

Erro quadrático médio:  53.44426628040599 
R2:  0.9911244491338851


 - O alto valor de R² indica que o modelo está bem ajustado e é capaz de prever com precisão o total de mortes com base nas variáveis selecionadas.
 - O uso de práticas padrão como divisão de dados, normalização, e validação durante o treinamento fortalece a robustez do modelo.